In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
np.random.seed(42)
random.seed(42)


In [ ]:
n_users = 120
n_movies = 25
n_genres = 5

users = [f"User_{i}" for i in range(n_users)]
movies = [f"Movie_{i}" for i in range(n_movies)]

In [ ]:
movie_genres = np.random.randint(0, n_genres, size=n_movies)

In [ ]:
user_preferences = np.random.rand(n_users, n_genres)

In [ ]:
ratings = []

for user_id in range(n_users):
    for movie_id in range(n_movies):
        
        # вероятность, что пользователь вообще посмотрел фильм
        if np.random.rand() < 0.4:  # 40% фильмов просмотрены
            
            genre = movie_genres[movie_id]
            
            # оценка зависит от предпочтения к жанру
            base_rating = user_preferences[user_id, genre] * 5
            
            noise = np.random.normal(0, 0.5)
            
            rating = np.clip(base_rating + noise, 1, 5)
            
            ratings.append([
                users[user_id],
                movies[movie_id],
                round(rating, 1)
            ])


In [ ]:
df = pd.DataFrame(ratings, columns=["user", "movie", "rating"])
df.head()

In [ ]:
user_movie_matrix = df.pivot_table(
    index="user",
    columns="movie",
    values="rating"
).fillna(0)


In [ ]:
similarity = cosine_similarity(user_movie_matrix)

similarity_df = pd.DataFrame(
    similarity,
    index=user_movie_matrix.index,
    columns=user_movie_matrix.index
)


In [ ]:
Теперь сделаем аккуратную функцию.
def recommend_movies(target_user, top_k_users=5, top_n_movies=5):
    
    # 1. Находим похожих пользователей
    sim_scores = similarity_df[target_user].sort_values(ascending=False)
    similar_users = sim_scores.iloc[1:top_k_users+1].index
    
    # 2. Получаем фильмы, которые смотрели они
    similar_users_ratings = user_movie_matrix.loc[similar_users]
    
    # 3. Усредняем оценки
    mean_ratings = similar_users_ratings.mean()
    
    # 4. Убираем фильмы, которые уже смотрел пользователь
    watched_movies = user_movie_matrix.loc[target_user]
    
    recommendations = mean_ratings[watched_movies == 0]
    
    # 5. Возвращаем топ
    return recommendations.sort_values(ascending=False).head(top_n_movies)


In [ ]:
recommend_movies("User_10")